In [2]:
# Import Libraries
from time import sleep
from random import randint
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [14]:
def imdb_extract(url, no_of_pages, get_partner):
    '''
    Method to scrape data from imdb website.
    
    url: url of imdb list (str)
    no_of_pages: number of desired pages (int)
    get_partner: whether get celebrity partner or not (bool)
    
    
    '''
    dict_of_marriages = {}
    for i in range(1,no_of_pages + 1):
        dict_of_links = {}
        dict_of_actors = {}
        ind = url.index('page')
        url = url[:ind] + 'page={i}'.format(i = i)
        print(i)
        response = get(url)
        sleep(randint(8,15))
        html_soup = BeautifulSoup(response.text, 'html.parser')
        actors_headers = html_soup.find_all('h3', class_ = 'lister-item-header')
        j = 0
        for actor in actors_headers:
            dict_of_links[actor.a.get_text().strip('\n')]  = ['https://www.imdb.com' + actor.a.get('href') ]
            j+=1
            bio = '/bio?ref_=nm_ov_bio_sm'
        for key, value in dict_of_links.items():
            print(key)
            to_get = value[0] + bio
            to_get_award = value[0]
            response = get(to_get)
            response_award = get(to_get_award)
            soup = BeautifulSoup(response.text, 'html.parser')
            soup_award = BeautifulSoup(response_award.text, 'html.parser')
            try:
                search_award = soup_award.find('span', class_ = 'awards-blurb').get_text()
                nom = ('Oscar' in search_award) or ('Golden Globe' in search_award) or ('BAFTA' in search_award )
            except:
                nom = False
            ovr = soup.find('table', id = 'overviewTable')
            try:
                spouse = soup.find('table', id = 'tableSpouses')
                birth_date = ovr.time.get('datetime')
            except AttributeError:
                continue
            mini_bio = soup.find('p')
            str_mini_bio = str(mini_bio)
            celeb_gender = 'Male' if str_mini_bio.lower().count(' he ') > str_mini_bio.lower().count(' she ') else 'Female'
            try:
                links = spouse.find_all('td')
            except AttributeError:
                continue
            no_of_links = len(spouse.find_all('td'))
            if get_partner == True:
                for i in range(no_of_links):
                    if i % 2 == 0:
                        flag = True
                        partner_name = links[i].get_text()
                        
                        try:
                            link = links[i].a.get('href')
                            url_p = 'https://www.imdb.com' + link + bio
                            url_p_award = 'https://www.imdb.com' + link
                        except  AttributeError:
                            status = links[i+1].get_text()
                            dict_of_marriages[partner_name + '/' + key] = [birth_date, -1, nom, False, status,
                                                         celeb_gender, -1]
                            flag = False 
                            continue
                        res_p = get(url_p)
                            res_p_award = get(url_p_award)
                            soup_p = BeautifulSoup(res_p.text, 'html.parser')
                        soup_p_award = BeautifulSoup(res_p_award.text, 'html.parser')
                        try:
                            search_award_p = soup_p_award.find('span', class_ = 'awards-blurb').get_text()
                            nom_p = ('Oscar' in search_award_p) or ('Golden Globe' in search_award_p) or ('BAFTA' in search_award_p)
                        except AttributeError:
                            nom_p = False
                        ovr_p = soup_p.find('table', id = 'overviewTable')
                        try:
                            container = soup_p.find('table', id = 'overviewTable')
                            birth_date_p = container.time.get('datetime')
                            #birth_date_p = ovr_p.time.get('datetime')    
                        except AttributeError:
                            birth_date_p = -1
                
                        mini_bio_p = soup_p.find('p')
                        str_mini_bio_p = str(mini_bio_p)
                        partner_gender = 'Male' if str_mini_bio_p.lower().count(' he ') > str_mini_bio_p.lower().count(' she ') else 'Female' if str_mini_bio_p.lower().count(' he ') < str_mini_bio_p.lower().count(' she ') else '-1'
                    elif flag == True:
                        status = links[i].get_text()
                        dict_of_marriages[partner_name + '/' + key] = [birth_date, birth_date_p, nom, nom_p, status,
                                                         celeb_gender, partner_gender]
            else:
                for i in range(1, no_of_links, 2):
                    status = links[i].get_text()
                dict_of_marriages[key] = [birth_date, nom, status, celeb_gender]
                
    return dict_of_marriages

In [20]:
dict_of_marriages = imdb_extract('https://www.imdb.com/list/ls021267673/?sort=list_order,asc&mode=detail&page=1', 2, True)

1
 Iwan Rheon
 Mae Whitman
 Jackie Earle Haley
 Helen McCrory
 Alexander Ludwig
 Charisma Carpenter
 Evan Peters
 Sarah Bolger
 Thomas Brodie-Sangster
 Emilia Fox
 Will Poulter
 Isabelle Fuhrman
 John Carroll Lynch
 Anika Noni Rose
 Jon Polito
 Samantha Morton
 Rufus Sewell
 Peta Wilson
 Eddie Marsan
 Nikki Blonsky
 Burn Gorman
 Fiona Dourif
 Ted Levine
 Jodie Whittaker
 Doug Jones
 Elizabeth Mitchell
 Jeremy Northam
 Lacey Chabert
 William Moseley
 Ashley Scott
 Xander Berkeley
 Kaya Scodelario
 Ben Daniels
 Kelly Macdonald
 Toby Jones
 Ivana Baquero
 Mark Gatiss
 JoAnna Garcia Swisher
 David Johansen
 Jamie Chung
 Jonathan Brandis
 Dina Meyer
 Avan Jogia
 Vanessa Morgan
 Jonathan Banks
 Annette O'Toole
 Brion James
 Rachel Hurd-Wood
 Mark Rylance
 Patricia Velasquez
 Dominic Cooper
 Melinda Clarke
 Sebastian Stan
 Holliday Grainger
 Ezra Miller
 Lysette Anthony
 Domhnall Gleeson
 Caitlin Stasey
 Douglas Booth
 Freema Agyeman
 Jason Clarke
 Tina Majorino
 Alan Tudyk
 Lara Pulver
 Shar

In [23]:
df = pd.DataFrame(dict_of_marriages.values(), index= dict_of_marriages.keys(),
                 columns = ['celeb_bd', 'partner_bd', 'celeb_nominated','partner_nominated', 'marriage_status', 'celeb_gender', 'partner_gender']   )
df.head()

,celeb_bd,partner_bd,celeb_nominated,partner_nominated,marriage_status,celeb_gender,partner_gender
Amelia Cruz / Jackie Earle Haley,1961-7-14,-1,True,False,\n(6 August 2004 - \npresent) ...,Male,-1
Jennifer Hargrave / Jackie Earle Haley,1961-7-14,-1,True,False,\n(1985 - \n?) ( divorced) ( ...,Male,-1
Sherry Vaughan / Jackie Earle Haley,1961-7-14,-1,True,False,\n(6 November 1979 - \n?) ( divo...,Male,-1
Damian Lewis / Helen McCrory,1968-8-17,1971-2-11,False,True,\n(4 July 2007 - \npresent) ( 2 ...,Female,Male
Damian Hardy / Charisma Carpenter,1970-7-23,-1,False,False,\n(5 October 2002 - \n8 July 2008) ...,Female,-1


In [24]:
df.to_csv('d22020.csv')